# Pipeline Tạo Bộ Dữ Liệu Prompt cho Agent Đánh giá Sách

Notebook này thực hiện một quy trình hoàn chỉnh để tự động tạo ra các prompt (câu lệnh) dùng để đánh giá sách. Quy trình bao gồm các bước sau:

1.  **Thiết lập & Cấu hình:** Khai báo các thư viện và biến cần thiết.
2.  **Tải và Lấy mẫu Dữ liệu:** Đọc danh sách sách từ file CSV và chọn ra một số lượng sách ngẫu nhiên để xử lý.
3.  **Tạo Yêu cầu hàng loạt (Batch Requests):** Với mỗi cuốn sách, tạo một yêu cầu để Gemini tự sinh ra một chủ đề đánh giá sách thú vị.
4.  **Thực thi & Chờ Tác vụ:** Gửi tất cả các yêu cầu lên Gemini API dưới dạng một tác vụ hàng loạt (batch job) và chờ cho đến khi hoàn tất.
5.  **Xử lý & Lưu Kết quả:** Tổng hợp dữ liệu sách ban đầu với các prompt được tạo ra và lưu kết quả cuối cùng vào một file JSON.


### 1. Thiết lập & Cấu hình


In [24]:
import os
import csv
import json
import random
import time
from dotenv import load_dotenv
from google import genai
from tqdm.notebook import tqdm

load_dotenv()

# --- CÁC THAM SỐ CẤU HÌNH ---
INPUT_CSV_FILE = "data/sampled_books_eng_400.csv"  # File chứa tên sách và mô tả
OUTPUT_JSON_FILE = "data/book_review_agent_prompts.json"  # File kết quả cuối cùng
BOOKS_TO_PROCESS = 100  # Số lượng sách muốn xử lý trong một lần chạy

os.makedirs(
    os.path.dirname(OUTPUT_JSON_FILE), exist_ok=True
)  # Tạo thư mục nếu chưa tồn tại

try:
    client = genai.Client()
    print("Kết nối tới Gemini API thành công.")
except Exception as e:
    print(
        f"Lỗi: Không thể kết nối tới Gemini API. Vui lòng kiểm tra API key. Chi tiết: {e}"
    )

./data
Lỗi: Không thể kết nối tới Gemini API. Vui lòng kiểm tra API key. Chi tiết: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.


### 2. Tải và Lấy mẫu Dữ liệu


In [ ]:
all_books_data = []
try:
    with open(INPUT_CSV_FILE, "r", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            all_books_data.append(
                {
                    "title": row.get("title", "Unknown Title"),
                    "description": row.get("description", "No Description Available"),
                }
            )
    print(f"Đã đọc thành công {len(all_books_data)} cuốn sách từ '{INPUT_CSV_FILE}'.")
except FileNotFoundError:
    print(
        f"Lỗi: Không tìm thấy file '{INPUT_CSV_FILE}'. Vui lòng kiểm tra lại đường dẫn."
    )
    all_books_data = []  # Gán lại là list rỗng để các cell sau không bị lỗi

# Lấy mẫu ngẫu nhiên từ danh sách đã đọc
if all_books_data:
    if len(all_books_data) > BOOKS_TO_PROCESS:
        sampled_books = random.sample(all_books_data, BOOKS_TO_PROCESS)
        print(f"Đã lấy ngẫu nhiên {len(sampled_books)} cuốn sách để xử lý.")
    else:
        sampled_books = all_books_data
        print(
            f"Số lượng sách ít hơn hoặc bằng {BOOKS_TO_PROCESS}. Xử lý toàn bộ {len(sampled_books)} cuốn sách."
        )
else:
    sampled_books = []

sampled_books

### 3. Tạo Yêu cầu hàng loạt (Batch Requests)


In [44]:
def focus_clear_prompt(title, description):
    return f"""
    Bạn là một "prompt generator" cho đánh giá sách.
    Viết **một prompt ngắn gọn** để đánh giá cuốn sách "{title}" tập trung vào một **chủ đề nổi bật, thú vị** của câu chuyện. 
    Mô tả ngắn gọn của sách: {description}
    
    Yêu cầu:
    - Chỉ trả về duy nhất một prompt, không giải thích hay liệt kê nhiều ví dụ.
    - Prompt phải rõ ràng, súc tích và gợi mở một góc nhìn phân tích sâu sắc.

    Ví dụ đầu ra mong muốn:
    Tôi muốn viết bài đánh giá về cuốn sách "Hoàng Tử Bé" tập trung vào sự đối lập giữa sự ngây thơ của trẻ con với sự khô khan, thực dụng của người lớn.
    """

In [43]:
batch_requests = []
if sampled_books:
    for book in sampled_books:
        title = book["title"]
        description = book["description"]

        prompt_template = focus_clear_prompt(title, description)

        request_obj = {
            "contents": [{"parts": [{"text": prompt_template}], "role": "user"}]
        }
        batch_requests.append(request_obj)

    print(f"Đã tạo thành công {len(batch_requests)} yêu cầu cho batch job.")
else:
    print("Không có sách nào để tạo yêu cầu.")

Đã tạo thành công 100 yêu cầu cho batch job.


### 4. Thực thi & Chờ Tác vụ


In [ ]:
batch_job = None
if batch_requests:
    print("Đang gửi batch job lên Gemini API...")
    batch_job = client.batches.create(model="gemini-2.0-flash", src=batch_requests)
    print(f"Đã tạo batch job thành công: {batch_job.name}")

    # Chờ cho tác vụ hoàn thành
    while batch_job.state.name not in (
        "JOB_STATE_SUCCEEDED",
        "JOB_STATE_FAILED",
        "JOB_STATE_CANCELLED",
        "JOB_STATE_EXPIRED",
    ):
        print(f"Trạng thái job: {batch_job.state.name}. Đang chờ 60 giây...")
        time.sleep(60)
        batch_job = client.batches.get(name=batch_job.name)

    print(f"Tác vụ đã hoàn thành với trạng thái: {batch_job.state.name}")
else:
    print("Không có yêu cầu nào để gửi đi.")

### 5. Xử lý & Lưu Kết quả


In [ ]:
final_dataset = []

if batch_job and batch_job.state.name == "JOB_STATE_SUCCEEDED":
    print("Đang xử lý và tổng hợp kết quả...")

    for i in tqdm(range(len(sampled_books)), desc="Đang tổng hợp"):
        original_book = sampled_books[i]
        response_item = batch_job.dest.inlined_responses[i]

        generated_prompt = "Lỗi: Không có phản hồi"
        if response_item.response and response_item.response.text:
            generated_prompt = response_item.response.text.strip()

        final_dataset.append(
            {
                "no": i + 1,
                "book_title": original_book["title"],
                "description": original_book["description"],
                "generated_prompt": generated_prompt,
            }
        )

    # Lưu kết quả ra file JSON
    with open(OUTPUT_JSON_FILE, "w", encoding="utf-8") as f:
        json.dump(final_dataset, f, indent=2, ensure_ascii=False)

    print(
        f"Hoàn tất! Đã lưu {len(final_dataset)} bản ghi vào file '{OUTPUT_JSON_FILE}'."
    )
else:
    print(
        "Tác vụ không thành công hoặc không có dữ liệu để xử lý. Sẽ không ghi file nào."
    )